# NLP - CA6
* **Name:** Mohammad Mahdi Salmani
* **Student id:** 810102174

## Load libraries

In [ ]:
%pip install -qU langchain\
    langchain-community\
    langchain-together\
    langchain-core\
    faiss-cpu\
    faiss-gpu\
    langgraph\
    sentence-transformers\
    gradio\
    pypdf

In [1]:
import os
import requests
from bs4 import BeautifulSoup
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from dotenv import load_dotenv
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_together import ChatTogether
from langchain_community.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Literal
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from IPython.core.display import Markdown
from langchain_core.messages import HumanMessage, AIMessage
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.embeddings import CacheBackedEmbeddings

## 1. Load dataset

In [2]:
os.environ["TAVILY_API_KEY"] = "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"
os.environ["TOGETHER_API_KEY"] = "92a237b154b842a8af1371f10bc4710f92c1be002b4ad6e5c5c29e6914304e42"

### Crawl pdf list from site

In [3]:
url = "https://stanford.edu/~jurafsky/slp3/"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

chapters = []
for link in soup.find_all('a', href=True):
    if '.pdf' in link['href'] and link['href'].split('.')[0].isdigit():
        chapters.append(link['href'])

pdf_links = [url+link for link in chapters]
pdf_links

['https://stanford.edu/~jurafsky/slp3/2.pdf',
 'https://stanford.edu/~jurafsky/slp3/3.pdf',
 'https://stanford.edu/~jurafsky/slp3/4.pdf',
 'https://stanford.edu/~jurafsky/slp3/5.pdf',
 'https://stanford.edu/~jurafsky/slp3/6.pdf',
 'https://stanford.edu/~jurafsky/slp3/7.pdf',
 'https://stanford.edu/~jurafsky/slp3/8.pdf',
 'https://stanford.edu/~jurafsky/slp3/9.pdf',
 'https://stanford.edu/~jurafsky/slp3/10.pdf',
 'https://stanford.edu/~jurafsky/slp3/11.pdf',
 'https://stanford.edu/~jurafsky/slp3/13.pdf',
 'https://stanford.edu/~jurafsky/slp3/14.pdf',
 'https://stanford.edu/~jurafsky/slp3/15.pdf',
 'https://stanford.edu/~jurafsky/slp3/16.pdf',
 'https://stanford.edu/~jurafsky/slp3/17.pdf',
 'https://stanford.edu/~jurafsky/slp3/18.pdf',
 'https://stanford.edu/~jurafsky/slp3/19.pdf',
 'https://stanford.edu/~jurafsky/slp3/20.pdf',
 'https://stanford.edu/~jurafsky/slp3/21.pdf',
 'https://stanford.edu/~jurafsky/slp3/22.pdf',
 'https://stanford.edu/~jurafsky/slp3/23.pdf']

### Download PDFs

In [4]:
documents = []
for pdf_link in pdf_links[:2]:
    loader = PyPDFLoader(pdf_link)
    documents.extend(loader.load())

In [5]:
len(documents)

60

### Split texts

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
chunks = text_splitter.split_documents(documents)

len(chunks)

203

### Embedder

In [8]:
# model_name = ""
# model_kwargs = {'device': 'cpu'}
# encode_kwargs = {'normalize_embeddings': False}
# embedding_function = HuggingFaceEmbeddings(
#     model_name=model_name,
#     model_kwargs=model_kwargs,
#     encode_kwargs=encode_kwargs
# )

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [19]:
# embedding_function = HuggingFaceEmbeddings(show_progress=True, multi_process=True)
cache_backed_embeddings = CacheBackedEmbeddings(embedding_function, cache_dir="embedding_cache")

TypeError: CacheBackedEmbeddings.__init__() got an unexpected keyword argument 'cache_dir'

In [ ]:
cache_file = "embedding_cache.json"

def load_cache():
    if os.path.exists(cache_file):
        with open(cache_file, "r") as f:
            return json.load(f)
    return {}

def save_cache(cache):
    with open(cache_file, "w") as f:
        json.dump(cache, f)

cache = load_cache()
cached_embedder = CacheBackedEmbeddings(base_embedder=embedder, cache=cache)

vectorstore = FAISS.from_texts(split_documents, cached_embedder)

vectorstore.save("faiss_index")

# ذخیره کردن Cache
save_cache(cached_embedder.cache)

# بارگذاری مجدد FAISS index (اختیاری)
# vectorstore = FAISS.load("faiss_index", cached_embedder)